In [1]:
# Python modules
import numpy as np
from pathlib import Path
import pyproj
import rasterio as rio
import sys

# local modules
sys.path.append("/home/nilscp/GIT/rastertools")
sys.path.append("/home/nilscp/GIT/download_planetary_datasets")

import utils
from Kaguya import global_imagery_and_DEM

### How to download all of the kaguya products for a specific area
----
A function in `get_links_for_download`, the function `get_main_repo_links`, will fetch all of the strings required to generate the url for download for a specified Kaguya product. A small example of the function is given below. 

Products available:
- product DTM_MAP_01, SLDEM2013 (1x1 degree)
- product TCO_MAP_02, visible orthoimages (3x3 degrees)
- product TCO_MAPe04, visible evening images (3x3 degrees)
- product TCO_MAPm04, visible morning images (3x3 degrees)

In [2]:
(default_link, default_link2, degrees_per_latlon) = global_imagery_and_DEM.get_main_repo_links("TCO_MAP_02")

In [3]:
default_link, default_link2, degrees_per_latlon

('http://darts.isas.jaxa.jp/pub/pds3/sln-l-tc-5-ortho-map-v2.0/',
 '/data/TCO_MAP_02_',
 3)

In [4]:
kaguya_products = ['DTM_MAP_01', 'TCO_MAP_02', 'TCO_MAPe04', 'TCO_MAPm04']

for products in kaguya_products:

    global_imagery_and_DEM.get_links_for_download(output_folder = '/home/nilscp/tmp/test_vrt/' + products + '/', 
                           product_name = products, 
                           min_latitude = -3, 
                           max_latitude = 3, 
                           min_longitude = 0, 
                           max_longitude = 3)

DONE
DONE
DONE
DONE


I need to improve the message printed here.

### Download files
----
Now that the url are saved to a text file, they can be easily downloaded using `wget -c -N -i ./<path_to_text_file>`

### Convert .img to .tif files
----
with the help of the `translate_kaguya.sh` bash script. Note that the wget, the translation and the vrt building could be put in the same bash file.

### Build the virtual mosaics (vrt) for each product 
----
by running `gdalbuildvrt <vrt filename> *.tif`

### Extract data for an area of interest with a bounding box containing world coordinates (lon/lat)
----
Note that in this example, we only have data in between 0 and 3 degrees in longitude, and -3 and 3 in latitude. Let's study a crater located at 0.39E and 1.16N (the Bruce impact crater, which is about 6.6 km in size. 

In [5]:
sldem2013 = Path('/home/nilscp/tmp/test_vrt/DTM_MAP_01/DTM_MAP_01.vrt')
TCO_MAP_02 = Path('/home/nilscp/tmp/test_vrt/TCO_MAP_02/TCO_MAP_02.vrt')
TCO_MAPe04 = Path('/home/nilscp/tmp/test_vrt/TCO_MAPe04/TCO_MAPe04.vrt')
TCO_MAPm04 = Path('/home/nilscp/tmp/test_vrt/TCO_MAPm04/TCO_MAPm04.vrt')

template_raster_sldem = Path('/home/nilscp/tmp/test_vrt/DTM_MAP_01/DTM_MAP_01_N00E000S01E001SC.tif')
template_raster_tco = Path('/home/nilscp/tmp/test_vrt/TCO_MAP_02/TCO_MAP_02_N00E000S03E003SC.tif')

Let's double check that there is no differences in the projection of products

In [6]:
with rio.open(sldem2013) as src_sldem2013:
    meta_sldem2013 = src_sldem2013.profile
    
with rio.open(TCO_MAP_02) as src_TCO_MAP_02:
    meta_TCO_MAP_02 = src_TCO_MAP_02.profile
    
with rio.open(TCO_MAPe04) as src_TCO_MAPe04:
    meta_TCO_MAPe04 = src_TCO_MAPe04.profile
    
with rio.open(TCO_MAPm04) as src_TCO_MAPm04:
    meta_TCO_MAPm04 = src_TCO_MAPm04.profile

In [7]:
meta_sldem2013

{'driver': 'VRT', 'dtype': 'int16', 'nodata': -32768.0, 'width': 12288, 'height': 24576, 'count': 1, 'crs': CRS.from_wkt('PROJCS["SIMPLE_CYLINDRICAL MOON",GEOGCS["GCS_MOON",DATUM["D_MOON",SPHEROID["MOON",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(7.4031617246699, 0.0, -3.70158086233495,
       0.0, -7.4031617246699, 90973.75285360607), 'blockxsize': 128, 'blockysize': 128, 'tiled': True}

In [8]:
meta_TCO_MAP_02

{'driver': 'VRT', 'dtype': 'uint16', 'nodata': 0.0, 'width': 12288, 'height': 24576, 'count': 1, 'crs': CRS.from_wkt('PROJCS["SIMPLE_CYLINDRICAL MOON",GEOGCS["GCS_MOON",DATUM["D_MOON",SPHEROID["MOON",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(7.4031617246699, 0.0, -3.70158086233495,
       0.0, -7.4031617246699, 90973.75285360607), 'blockxsize': 128, 'blockysize': 128, 'tiled': True}

In [9]:
meta_TCO_MAPe04 == meta_TCO_MAP_02 == meta_TCO_MAPm04

True

The tiling is different in `sldem2013`, so that the vrt file is different (but the proj. is the same) 

In [10]:
meta_sldem2013 == meta_TCO_MAP_02

False

In [11]:
meta_sldem2013['crs'] == meta_TCO_MAP_02['crs']

True

Let's define the template raster for saving the final products for the Bruce impact crater
### Specify the bounding box

In [12]:
bbox = utils.bbox_world2crs(src_sldem2013, [-0.10, 0.60, 0.90, 1.70]) #xmin, ymin, xmax, ymax
bbox

[-3032.335042414948,
 18194.010254489687,
 27291.015381734534,
 51549.695721054115]

### Reading

In [13]:
array_sldem2013 = utils.read(sldem2013, bbox)
array_TCO_MAP_02 = utils.read(TCO_MAP_02, bbox)
array_TCO_MAPe04 = utils.read(TCO_MAPe04, bbox)
array_TCO_MAPm04 = utils.read(TCO_MAPm04, bbox)

In [14]:
array_sldem2013.shape, array_TCO_MAP_02.shape, array_TCO_MAPe04.shape, array_TCO_MAPm04.shape

((4505, 3686, 1), (4505, 3686, 1), (4505, 3686, 1), (4505, 3686, 1))

### Saving to a raster

In [15]:
utils.to_raster(array_sldem2013, template_raster_sldem, bbox, '/home/nilscp/tmp/test_vrt/Bruce_crater/sldem2013.tif')
utils.to_raster(array_TCO_MAP_02, template_raster_tco, bbox, '/home/nilscp/tmp/test_vrt/Bruce_crater/TCO_MAP_02.tif')
utils.to_raster(array_TCO_MAPe04, template_raster_tco, bbox, '/home/nilscp/tmp/test_vrt/Bruce_crater/TCO_MAPe04.tif')
utils.to_raster(array_TCO_MAPm04, template_raster_tco, bbox, '/home/nilscp/tmp/test_vrt/Bruce_crater/TCO_MAPm04.tif')

I guess mineralogy data from Lemelin et al. could also be used. **The only problem is that there is a quite a shift differences between JAXA and NASA type of data.** So if possible, I would say that it is better to stick to one another. For very small craters (D < 1 km), LRO NAC is the best. Everything between 1 < D < 15 km, I would say that Kaguya is the best. For craters larger than that (D > 15-30 km), the WAC global 100 m image will do the trick. The only problem is that a very small amount of NAC DTM have been generated, and the process to generate NAC DTM was not that easy a few years ago. It's maybe much easier now. So if DTM data needs to be extracted (for D < 1 km), using Kaguya image is probably advised.    